In [ ]:
import sys
import os
# Append the library path to PYTHONPATH, so library can be imported.
sys.path.append(os.path.dirname(os.getcwd()))
import itertools

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from library import common as cm

In [ ]:
%run setup.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

sns.set(style='darkgrid')

In [ ]:
cols = ['CONFIG=1/FREQ=1D_HALFMONEY=otm_MINM=0.8_MAXM=1.5_Permute=False_VIX=False',
       'CONFIG=1/FREQ=1D_HALFMONEY=otm_MINM=0.8_MAXM=1.5_Permute=False_VIX=True',
       'CONFIG=1/FREQ=1D_HALFMONEY=otm_MINM=0.8_MAXM=1.5_Permute=True_VIX=False',
       'CONFIG=1/FREQ=1D_HALFMONEY=otm_MINM=0.8_MAXM=1.5_Permute=True_VIX=True']

In [ ]:
rows = ['Regression/BS_Benchmark', 
        'Regression/Delta_Vega_Vanna',
       'Network/Delta_Vega']

In [ ]:
df_dirs = pd.DataFrame(columns=cols, index=rows)
for x, y in list(itertools.product(rows, cols)):
    df_dirs.loc[x, y] = f'{DATA_DIR}Result/{y}/{x}/'

In [ ]:
DATA_DIR

In [ ]:
kws = [('Regression/BS_Benchmark', 'Black-Scholes', 'o', -0.1, sns.color_palette()[0]),
       ('Regression/Delta_Vega_Vanna', 'Delta-Vega-Vanna regression', 'x', -0.05, sns.color_palette()[1]),
       ('Network/Delta_Vega',  r'ANN $(\Delta_{\rm BS};\, \mathcal{V}_{\rm BS};\, \tau)$', '^', 0.1, sns.color_palette()[3])]

In [ ]:
aggregating = 'mean'
measure = 'mse'

In [ ]:
inspector = cm.Inspector()

In [ ]:
df_res = inspector.evalPnls(df_dirs, aggregating=aggregating, measure=measure, op_type=None)
res_abs = df_res.xs('Absolute', level=1, axis=1)
df_res

In [ ]:
for col in kws:
    plt.plot(np.arange(1, 5)+col[3], res_abs.loc[col[0]], marker=col[2], linewidth=0, label=col[1], c=col[4])
plt.legend(frameon=False)
plt.xticks(np.arange(1, 5), ('Baseline', 'VIX', 'Permute', 'Permute + VIX'))
plt.ylabel('Mean squared hedging error')
plt.title('Absolute value')

In [ ]:
res_rela = df_res.xs('%Change', level=1, axis=1)

In [ ]:

for col in kws[1:]:
    plt.plot(np.arange(1, 5)+col[3], res_rela.loc[col[0]], marker=col[2], linewidth=0, label=col[1], c=col[4])
plt.legend(frameon=False)
plt.xticks(np.arange(1, 5), ('Baseline', 'VIX', 'Permute', 'Permute + VIX'))
plt.ylabel('Relative reduction to Black-Scholes Delta')
plt.title('Black-Scholes (1 day)')

#### Baseline
This section shows the the MSHE and its relative percentage change of the two statistical models compared to the BS-Delta in the 'baseline' experiment.

In [ ]:
dirs_dict = {'Regression/BS_Benchmark': f'{DATA_DIR}Result/{cols[0]}/Regression/BS_Benchmark/',
             'Regression/Delta_Vega_Vanna': f'{DATA_DIR}Result/{cols[0]}/Regression/Delta_Vega_Vanna/',
            'Network/Delta_Vega': f'{DATA_DIR}Result/{cols[0]}/Network/Delta_Vega/'}

In [ ]:
inspector = cm.Inspector()
baseline = inspector.eval_single_exp(dirs_dict, measure=measure)

In [ ]:
bs_tmp = baseline['Regression/BS_Benchmark'].values[:, np.newaxis]
base_rela = (baseline[['Regression/Delta_Vega_Vanna', 'Network/Delta_Vega']] - bs_tmp) / bs_tmp

In [ ]:
for col in kws[1:]:
    plt.plot(np.arange(1, NUM_TEST+1)+col[3], base_rela.loc[:, col[0]]*100, 
             marker=col[2], linewidth=0, label=col[1], c=col[4])
plt.legend(frameon=False)
plt.ylabel('Mean squared hedging error')
plt.title('Relative to Black-Scholes')

#### 'VIX' experiment
This section shows the the MSHE and its relative percentage change of the two statistical models compared to the BS-Delta in the 'VIX' experiment, on the ten out-of-sample sets

In [ ]:
dirs_dict = {'Regression/BS_Benchmark': f'{DATA_DIR}Result/{cols[1]}/Regression/BS_Benchmark/',
             'Regression/Delta_Vega_Vanna': f'{DATA_DIR}Result/{cols[1]}/Regression/Delta_Vega_Vanna/',
            'Network/Delta_Vega': f'{DATA_DIR}Result/{cols[1]}/Network/Delta_Vega/'}
inspector = cm.Inspector()
res = inspector.eval_single_exp(dirs_dict, measure=measure)

In [ ]:
for col in kws[1:]:
    plt.plot(np.arange(1, NUM_TEST+1)+col[3], res.loc[:, col[0]], marker=col[2], linewidth=0, label=col[1], c=col[4])
plt.legend(frameon=False)
plt.ylabel('Mean squared hedging error')
plt.title('Absolute value')

In [ ]:
for col in kws[1:]:
    y = (res[col[0]] - res['Regression/BS_Benchmark']) / res['Regression/BS_Benchmark']
    plt.plot(np.arange(1, NUM_TEST+1)+col[3], y, marker=col[2], linewidth=0, label=col[1], color=col[4])
plt.legend(frameon=False)
plt.ylabel('Percentage')
plt.title('Relative')

#### 'Permute' experiment
This section shows the the MSHE and its relative percentage change of the two statistical models compared to the BS-Delta in the 'Permutation' experiment.

In [ ]:
dirs_dict = {'Regression/BS_Benchmark': f'{DATA_DIR}Result/{cols[2]}/Regression/BS_Benchmark/',
             'Regression/Delta_Vega_Vanna': f'{DATA_DIR}Result/{cols[2]}/Regression/Delta_Vega_Vanna/',
            'Network/Delta_Vega': f'{DATA_DIR}Result/{cols[2]}/Network/Delta_Vega/'}
inspector = cm.Inspector()
res = inspector.eval_single_exp(dirs_dict, measure=measure)

In [ ]:
for col in kws[1:]:
    plt.plot(np.arange(1, NUM_TEST+1)+col[3], res.loc[:, col[0]], marker=col[2], linewidth=0, label=col[1], c=col[4])
#plt.legend(frameon=False)
plt.ylabel('Mean squared hedging error')
plt.title('Absolute value')

In [ ]:
" Relative to the BS "
bs_tmp = res['Regression/BS_Benchmark'].values[:, np.newaxis]
res_rela = (res[['Regression/Delta_Vega_Vanna', 'Network/Delta_Vega']] - bs_tmp) / bs_tmp
for col in kws[1:]:
    y = (res[col[0]] - res['Regression/BS_Benchmark']) / res['Regression/BS_Benchmark']
    plt.plot(np.arange(1, NUM_TEST+1)+col[3], y*100, marker=col[2], linewidth=0, label=col[1], color=col[4])
plt.legend(frameon=False)
plt.ylabel('Percentage improvement')
plt.title('Relative')

In [ ]:
" This plots shows the change of percentage improvemen of the two models  comapred to the baseline experiment"
tmp = res_rela - base_rela

for x in kws[1:]:
    plt.plot(range(1, NUM_TEST+1), tmp[x[0]] * 100, marker=x[2], linewidth=0, label=x[1], c=x[4])
plt.xlabel('Permutation set')
plt.ylabel('Percentage improvement')
plt.legend(frameon=False)
plt.title('Black-Scholes')

rela_diff_pmt = tmp

#### Permuate + VIX experiment

In [ ]:
dirs_dict = {'Regression/BS_Benchmark': f'{DATA_DIR}Result/{cols[3]}/Regression/BS_Benchmark/',
             'Regression/Delta_Vega_Vanna': f'{DATA_DIR}Result/{cols[3]}/Regression/Delta_Vega_Vanna/',
            'Network/Delta_Vega': f'{DATA_DIR}Result/{cols[3]}/Network/Delta_Vega/'}
inspector = cm.Inspector()
res = inspector.eval_single_exp(dirs_dict, measure=measure)

In [ ]:
for col in kws:
    plt.plot(np.arange(1, NUM_TEST+1)+col[3], res.loc[:, col[0]], marker=col[2], linewidth=0, label=col[1], c=col[4])
#plt.legend(frameon=False)
plt.ylabel('Mean squared hedging error')
plt.title('Absolute value')

In [ ]:
" Improvement compared to the Black-Scholes "
bs_tmp = res['Regression/BS_Benchmark'].values[:, np.newaxis]
res_rela = (res[['Regression/Delta_Vega_Vanna', 'Network/Delta_Vega']] - bs_tmp) / bs_tmp
for col in kws[1:]:
    y = (res[col[0]] - res['Regression/BS_Benchmark']) / res['Regression/BS_Benchmark']
    plt.plot(np.arange(1, NUM_TEST+1)+col[3], y*100, marker=col[2], linewidth=0, label=col[1], color=col[4])
plt.legend(frameon=False)
plt.ylabel('Percentage improvement')
plt.title('Relative')

In [ ]:
" the change of the percentage improvement  compared to the Baseline experiment"
tmp = res_rela - base_rela

for x in kws[1:]:
    plt.plot(range(1, NUM_TEST+1), tmp[x[0]] * 100, marker=x[2], linewidth=0, label=x[1], c=x[4])
plt.xlabel('Permutation set')
plt.ylabel('Percentage improvement')
plt.legend(frameon=False)
plt.title('Black-Scholes')

rela_diff_pmt_vix = tmp

In [ ]:
""" 
Combined plot of Permute and Permute + VIX, relative to baseline 
The permutation sets are sorted, according to the performance in the Baseline experiment.
"""
index_sorted = base_rela.sort_values(by='Network/Delta_Vega', ascending=False).index

fig, ax = plt.subplots()

kws_tmp = [('Regression/Delta_Vega_Vanna', 'DVV regr. (Permute)', 'x', -0.1),
 ('Network/Delta_Vega', 'ANN (Permute)', '^', -0.1)]

for x in kws_tmp:
    ax.plot(np.arange(1, NUM_TEST+1) + x[3], rela_diff_pmt.loc[index_sorted, x[0]].values*100, label=x[1], marker=x[2], linewidth=0)

kws_tmp = [('Regression/Delta_Vega_Vanna', 'DVV regr. (Permute + VIX)', 'x', 0.1),
         ('Network/Delta_Vega', 'ANN (Permute + VIX)', '^', 0.1)]  
for x in kws_tmp:
    ax.plot(np.arange(1, NUM_TEST+1) + x[3], rela_diff_pmt_vix.loc[index_sorted, x[0]].values*100, label=x[1], marker=x[2], linewidth=0)
plt.xticks([1, 5, 10, 15, 20], [1, 5, 10, 15, 20])
plt.legend(frameon=False)
plt.xlabel('Permutation set')
plt.ylabel('Difference to baseline in relative reduction')
plt.title("Black-Scholes (1 day)")